<h1 style="text-align:center">IBM Data Science - CAPSTONE PROJECT<h1>

## Table of content

<div class="alert alert-block alert-info" style="margin-top: 20px">

1. [Retrieve the data from Wikipedia page](#retrieve_wikipedia)
2. [Data Wrangling](#data_wrangling)
3. [Exploration of the neighborhoods in Toronto](#exploration)
4. [Analysis of Each Neighborhood](#analysis_neighborhood)
5. [Cluster Neighborhoods](#cluster_neighborhoods)
6. [Examine Clusters](#clusters)

</div>
<hr>

Import all the libraries needed

In [1]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Retrieve the data from Wikipedia page <a id="retrieve_wikipedia"></a>

Let's first scrape our wikipedia page using __BeautifulSoup__.

In [2]:
import requests
postal_code_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(postal_code_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"a5871040-459e-43b7-b05e-bf97177c9687","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

Find class ‘wikitable sortable’ in the HTML script.

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})

Create the dataframe from html

In [5]:
df = pd.read_html(str(My_table))[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## 2. Data Wrangling <a id="data_wrangling"></a>

Drop the cells with a borough that is 'Not assigned'

In [6]:
df.replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we droped rows
df.reset_index(drop=True, inplace=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<p>Here the Neighborhoods having the same Postal Code are already in the same raw, and separated with a coma.</p>
<p>Let's check that there are no missing values anymore :<p>

In [7]:
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

Postal Code
False    103
Name: Postal Code, dtype: int64

Borough
False    103
Name: Borough, dtype: int64

Neighborhood
False    103
Name: Neighborhood, dtype: int64



OK, no missing values in our dataset !

In [8]:
print ("The size of the new dataset is: {} x {}".format(df.shape[0], df.shape[1]))

The size of the new dataset is: 103 x 3


In [9]:
# print a profile report of the dataset
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof.to_file(output_file='report.html')

In order to retrieve the latitude and the longitude from the postal code, we will use the library __pgeocode__ :

In [10]:
import pgeocode

# we want to retrieve the latitude/longitude from a postal code in Canada 'ca'
nomi = pgeocode.Nominatim('ca')

latitude = []
longitude = []

for index, row in df.iterrows():
    location = nomi.query_postal_code(row[0])  # row[0] represents Postal Code value
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    
# we put the result of the loop in new columns 'latitude' and 'longitude'
df['Latitude'] = latitude
df['Longitude'] = longitude


# pb with Canada Post Gateway Processing Centre > need to do the query manually
df.loc[df['Neighborhood'] == "Canada Post Gateway Processing Centre", ['Latitude', 'Longitude']] = [43.636966,-79.615819]

## 3. Exploration of the neighborhoods in Toronto <a id="exploration"></a>

### Create a map of Toronto with neighborhoods superimposed on top

Get the latitude and longitude of Toronto :

In [11]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_long = location.longitude

In [12]:
# create map of TOronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_toronto)  
    
map_toronto

### Explore Neighborhoods in Toronto

#### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'FNRS21YKKOHZNCXE2IVQX3GCUSVYOUXZKLHTG1FZR2EK2C0K' # your Foursquare ID
CLIENT_SECRET = 'CG5K5YGHICCDTMTAAW441NHK5XPBHNR0FUKQI4BVROGIX4BE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FNRS21YKKOHZNCXE2IVQX3GCUSVYOUXZKLHTG1FZR2EK2C0K
CLIENT_SECRET:CG5K5YGHICCDTMTAAW441NHK5XPBHNR0FUKQI4BVROGIX4BE


#### Let's explore the first neighborhood in our dataframe

In [14]:
# Get the neighborhood's latitude and longitude values
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7545, -79.33.


#### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters

First, let's create the GET request URL :

In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FNRS21YKKOHZNCXE2IVQX3GCUSVYOUXZKLHTG1FZR2EK2C0K&client_secret=CG5K5YGHICCDTMTAAW441NHK5XPBHNR0FUKQI4BVROGIX4BE&v=20180605&ll=43.7545,-79.33&radius=500&limit=100'

Let's send the GET request and examine the resutls

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0b321d3ff7134271cb0169'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7590000045, 'lng': -79.32378161085641},
   'sw': {'lat': 43.7499999955, 'lng': -79.33621838914358}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label'

 We know that all the information is in the *items* key. Before we proceed, let's create the **get_category_type** function :

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,TTC stop #8380,Bus Stop,43.752672,-79.326351
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


### Exploration of all the neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [21]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

#### Let's check the size of the resulting dataframe

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(2180, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.7276,-79.3148,Portugril,43.725819,-79.312785,Portuguese Restaurant


## 4. Analysis of Each Neighborhood <a id="analysis_neighborhood"></a>

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,York Mills West,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2            Badminton Court  0.25
3               Skating Rink  0.25
4         Miscellaneous Shop  0.00


----Alderwood, Long Branch----
               venue  freq
0        Pizza Place  0.12
1        Coffee Shop  0.12
2     Sandwich Place  0.12
3           Pharmacy  0.12
4  Convenience Store  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Pizza Place  0.17
1                Coffee Shop  0.17
2  Middle Eastern Restaurant  0.17
3   Mediterranean Restaurant  0.17
4        Fried Chicken Joint  0.17


----Bayview Village----
                 venue  freq
0                 Park  0.25
1                Trail  0.25
2          Flower Shop  0.25
3            Locksmith  0.25
4  Monument / Landmark  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.08
1      Sa

                        venue  freq
0  Construction & Landscaping  0.67
1         Rental Car Location  0.33
2                 Yoga Studio  0.00
3         Moroccan Restaurant  0.00
4                 Music Venue  0.00


----Humberlea, Emery----
                       venue  freq
0                  Nightclub  0.17
1              Grocery Store  0.17
2                Coffee Shop  0.17
3  Latin American Restaurant  0.17
4             Discount Store  0.17


----Humewood-Cedarvale----
           venue  freq
0   Tennis Court  0.12
1    Coffee Shop  0.12
2  Deli / Bodega  0.12
3  Grocery Store  0.12
4          Field  0.12


----India Bazaar, The Beaches West----
              venue  freq
0    Ice Cream Shop  0.05
1      Liquor Store  0.05
2        Board Shop  0.05
3  Sushi Restaurant  0.05
4           Brewery  0.05


----Islington Avenue, Humber Valley Village----
           venue  freq
0       Pharmacy  0.33
1           Bank  0.17
2  Grocery Store  0.17
3           Park  0.17
4   Skating Rink  

                       venue  freq
0             Sandwich Place  0.09
1                       Café  0.09
2  Middle Eastern Restaurant  0.04
3             History Museum  0.04
4                   Pharmacy  0.04


----The Beaches----
               venue  freq
0                Pub  0.25
1        Cheese Shop  0.12
2             Bakery  0.12
3          Gastropub  0.12
4  Health Food Store  0.12


----The Danforth West, Riverdale----
                venue  freq
0    Greek Restaurant  0.21
1      Ice Cream Shop  0.06
2                Café  0.06
3          Restaurant  0.06
4  Italian Restaurant  0.06


----The Kingsway, Montgomery Road, Old Mill North----
              venue  freq
0  Sushi Restaurant  0.12
1    Breakfast Spot  0.12
2              Bank  0.12
3            Lounge  0.06
4       Coffee Shop  0.06


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.10
1  Afghan Restaurant  0.06
2        Coffee Shop  0.06
3               Bank  0.06
4     Sandwich Place  0.0

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Badminton Court,Breakfast Spot,Latin American Restaurant,Skating Rink,Women's Store,Fish & Chips Shop,Farmers Market,Fast Food Restaurant,Field,Flea Market
1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Convenience Store,Pub,Coffee Shop,Sandwich Place,Gym,Pharmacy,Health Food Store,Hawaiian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Deli / Bodega,Fried Chicken Joint,Middle Eastern Restaurant,Coffee Shop,Mediterranean Restaurant,Dance Studio,Falafel Restaurant,Food Court,Cuban Restaurant
3,Bayview Village,Flower Shop,Park,Locksmith,Trail,Women's Store,Field,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Grocery Store,Pizza Place,Liquor Store,Thai Restaurant,Greek Restaurant


## 5. Cluster Neighborhoods <a id="cluster_neighborhoods"></a>

Run *k*-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with df (toronto data) to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Drop "na" values
toronto_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)

# Cast Cluster Labels to int
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype("int64") 

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,3,Food & Drink Shop,Bus Stop,Park,Women's Store,Electronics Store,Food Court,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
1,M4A,North York,Victoria Village,43.7276,-79.3148,1,Pizza Place,Park,Coffee Shop,Intersection,Portuguese Restaurant,Hockey Arena,French Restaurant,Flea Market,Flower Shop,Fish Market
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,1,Coffee Shop,Breakfast Spot,Theater,Distribution Center,Pub,Electronics Store,Restaurant,Event Space,Food Truck,Spa
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,1,Clothing Store,Coffee Shop,Restaurant,Women's Store,Food Court,Furniture / Home Store,Sushi Restaurant,Shoe Store,Cosmetics Shop,Toy / Game Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,1,Gym,Coffee Shop,Park,Martial Arts Dojo,Burrito Place,Mexican Restaurant,Bubble Tea Shop,Ramen Restaurant,Sushi Restaurant,Beer Bar


Finally, let's visualize the resulting clusters

In [30]:
# create map
map_clusters = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters <a id="clusters"></a>

### Cluster 0

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,0,Home Service,Women's Store,Electronics Store,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


### Cluster 1

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Pizza Place,Park,Coffee Shop,Intersection,Portuguese Restaurant,Hockey Arena,French Restaurant,Flea Market,Flower Shop,Fish Market
2,Downtown Toronto,1,Coffee Shop,Breakfast Spot,Theater,Distribution Center,Pub,Electronics Store,Restaurant,Event Space,Food Truck,Spa
3,North York,1,Clothing Store,Coffee Shop,Restaurant,Women's Store,Food Court,Furniture / Home Store,Sushi Restaurant,Shoe Store,Cosmetics Shop,Toy / Game Store
4,Downtown Toronto,1,Gym,Coffee Shop,Park,Martial Arts Dojo,Burrito Place,Mexican Restaurant,Bubble Tea Shop,Ramen Restaurant,Sushi Restaurant,Beer Bar
5,Etobicoke,1,Pharmacy,Bank,Grocery Store,Park,Skating Rink,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Steakhouse,American Restaurant,Salad Place,Asian Restaurant
98,Etobicoke,1,Sushi Restaurant,Breakfast Spot,Bank,Coffee Shop,Smoke Shop,Burger Joint,Dessert Shop,Lounge,Bar,Liquor Store
99,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Yoga Studio,Diner,Mediterranean Restaurant,Men's Store,Hotel
100,East Toronto,1,Restaurant,Coffee Shop,Yoga Studio,Italian Restaurant,Breakfast Spot,Bookstore,Sushi Restaurant,Japanese Restaurant,Bank,Department Store


### Cluster 2

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Scarborough,2,Auto Garage,Women's Store,Fish & Chips Shop,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Electronics Store


### Cluster 3

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Food & Drink Shop,Bus Stop,Park,Women's Store,Electronics Store,Food Court,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
7,North York,3,Park,Gym,Construction & Landscaping,Home Service,River,Pool,Women's Store,Farmers Market,Electronics Store,Ethiopian Restaurant
13,North York,3,Park,Gym,Construction & Landscaping,Home Service,River,Pool,Women's Store,Farmers Market,Electronics Store,Ethiopian Restaurant
21,York,3,Park,Women's Store,Gym,Sporting Goods Shop,Mexican Restaurant,Beer Store,Bakery,Department Store,Dessert Shop,Food Court
27,North York,3,Residential Building (Apartment / Condo),Park,Women's Store,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
32,Scarborough,3,Grocery Store,Park,Eastern European Restaurant,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field
35,East York,3,Convenience Store,Intersection,Park,Women's Store,Fish & Chips Shop,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
36,Downtown Toronto,3,Music Venue,Harbor / Marina,Park,Café,Field,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
39,North York,3,Flower Shop,Park,Locksmith,Trail,Women's Store,Field,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
61,Central Toronto,3,Photography Studio,Park,Women's Store,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market


### Cluster 4

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,4,Construction & Landscaping,Rental Car Location,Women's Store,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
